<a href="https://colab.research.google.com/github/CMccormick0003/deep-learning-challenge/blob/main/AlphabetSoupCharity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
print(application_df.shape)
print(application_df.columns)
application_df.head()

## Preprocessing

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [ ]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
print(unique_counts)

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
print(application_df['APPLICATION_TYPE'].value_counts())

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(app_type_counts[app_type_counts < 51].index)
print(application_types_to_replace)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts_greater_than_one = classification_counts[classification_counts > 1]
print(classification_counts_greater_than_one)

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 95].index)
print(classifications_to_replace)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
# create a list of columns to encode
cat_cols = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']

# use pd.get_dummies to encode categorical variables
encoded_df = pd.get_dummies(application_df, columns=cat_cols)
encoded_df.shape

In [ ]:
# Split our preprocessed data into our features and target arrays
X = encoded_df.drop("IS_SUCCESSFUL", axis=1).values
y = encoded_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = Sequential()

# First hidden layer
nn.add(Dense(units=80, activation='relu', input_dim=51))

# Add 6 additional hidden layers
for i in range(6):
    nn.add(Dense(units=50, activation='relu'))

# Last hidden layer
nn.add(Dense(units=30, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
from tensorflow.keras.optimizers import SGD

sgd = SGD(learning_rate=0.1)

nn.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])


# Define the checkpoint path and filename
checkpoint_path = "model_weights/checkpoint_weights.{epoch:02d}.h5"

# Create a callback that saves the model's weights every five epochs
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq=5 * len(X_train_scaled),
    verbose=1)

# Train the model with checkpoint callback
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint_callback])

In [ ]:
# Evaluate the model using the test data
evaluation = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {evaluation[0]}, Accuracy: {evaluation[1]}")

In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharityOptimised.h5")

In [ ]:
# See the path of the current working directory 
import os
print(os.getcwd())